In [1]:
import pandas as pd
import numpy as np
import joblib
import math
from keras.models import load_model

import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# 전처리 끝낸 데이터
df_train = joblib.load('df_train.pkl')
df_test = joblib.load('df_test.pkl')


# XGR

In [ ]:
# 이동평균을 만들어서 해보자

In [ ]:
df_train

In [21]:
df_train['dew_point'].value_counts()

-19.262075    124
-19.380396    120
-19.376579    117
-19.258258    112
-19.368945    110
             ... 
-19.291212      1
-19.294535      1
-19.295026      1
-19.280482      1
-19.494916      1
Name: dew_point, Length: 279352, dtype: int64

In [11]:
df_train

,year,month,day,time,minute,stn_id,ws10_deg,ws10_ms,ta,re,hm,sun10,ts,dew_point,vis1,class
895952,0,1,1,0,10,4,0.878333,0.161290,0.273239,0.0,0.163113,0.0,0.075724,-19.694138,20000.0,4
895953,0,1,1,0,20,4,0.970000,0.236559,0.273239,0.0,0.165245,0.0,0.080178,-19.693712,20000.0,4
895954,0,1,1,0,30,4,0.939722,0.215054,0.273239,0.0,0.168443,0.0,0.075724,-19.693072,20000.0,4
895955,0,1,1,0,40,4,0.983056,0.182796,0.270423,0.0,0.174840,0.0,0.075724,-19.694609,20000.0,4
895956,0,1,1,0,50,4,0.982500,0.247312,0.267606,0.0,0.170576,0.0,0.075724,-19.698279,20000.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156455,2,12,31,23,10,4,0.848056,0.827957,0.416901,0.0,0.470149,0.0,0.089087,-19.489069,20000.0,4
3156456,2,12,31,23,20,4,0.816111,0.612903,0.411268,0.0,0.469083,0.0,0.080178,-19.494916,20000.0,4
3156457,2,12,31,23,30,4,0.761667,0.526882,0.408451,0.0,0.478678,0.0,0.077951,-19.495814,20000.0,4
3156458,2,12,31,23,40,4,0.750833,0.494624,0.405634,0.0,0.486141,0.0,0.073497,-19.497138,20000.0,4


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [13]:
# 시정 구간 분류 함수 정의
def classify_visibility(vis1):
    if vis1 < 200:
        return 1
    elif vis1 < 500:
        return 2
    elif vis1 < 1000:
        return 3
    else:
        return 4

In [18]:
# Feature와 Target 데이터 나누기
X_features = df_train.columns.difference(['year','month','day','time','minute','vis1', 'class']).values
y_vis1 = df_train['vis1'].values
y_class = df_train['class'].values

# 각 모델 초기화
models = [
    SVR(kernel='rbf', C=100, gamma=0.1),
    RandomForestRegressor(n_estimators=50, random_state=42),
    XGBRegressor(objective='reg:squarederror', random_state=42),
    GradientBoostingRegressor(random_state=42)
]

# Stacking에 사용할 데이터 초기화
stacking_predictions_vis1 = np.zeros((len(y_vis1), len(models)))  # vis1 예측용 각 모델의 예측값 저장할 배열 초기화
stacking_predictions_class = np.zeros((len(y_class), len(models)))  # class 예측용 각 모델의 예측값 저장할 배열 초기화

# K-fold cross-validation을 통한 Stacking 구현
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, val_index in kf.split(X_features):
    X_features_train, X_features_val = X_features[train_index], X_features[val_index]
    y_vis1_train, y_vis1_val = y_vis1[train_index], y_vis1[val_index]
    y_class_train, y_class_val = y_class[train_index], y_class[val_index]

    for i, model in enumerate(models):
        # vis1 예측
        model.fit(X_features_train, y_vis1_train)
        y_vis1_pred = model.predict(X_features_val)
        stacking_predictions_vis1[val_index, i] = y_vis1_pred

        # class 예측
        class_pred = np.array([classify_visibility(val) for val in y_vis1_pred])
        stacking_predictions_class[val_index, i] = class_pred

# Stacking 결과를 사용하여 Meta 모델 학습 (class 예측용)
meta_model_class = RandomForestRegressor(n_estimators=50, random_state=42)
meta_model_class.fit(stacking_predictions_vis1, y_class)

# 최종 예측값 계산 및 평가
final_predictions_vis1 = np.mean(stacking_predictions_vis1, axis=1)  # vis1 예측값의 평균을 최종 예측값으로 사용
final_predictions_class = meta_model_class.predict(stacking_predictions_vis1)  # Meta 모델로 class 예측값 계산
csi = np.mean(y_class == final_predictions_class)  # Classification Success Index 계산

print(f"CSI: {csi:.4f}")

ValueError: could not convert string to float: 'dew_point'

# 모델 합치기

In [ ]:
#### 여기까지하고 지희한테 보내기

In [107]:
# fog_test.csv부르기
df_original = pd.read_csv("fog_test.csv", index_col=1)
df_original['fog_test.class'].replace(-99, float('nan'), inplace=True)

In [108]:
# pred_A = pd.read_csv('C:\\Users\\Hong_PC\\Documents\\카카오톡 받은 파일\\240371.csv', index_col=0)
pred_C = pd.read_csv('C:\\Users\\Hong_PC\\Documents\\카카오톡 받은 파일\\240371_2번지역.csv', index_col=0)

In [109]:
# 지역 주의, 예측된 class 값만 df_original에 반영
# df_original.loc[df_original['fog_test.stn_id'].str[0] == 'A', 'fog_test.class'] = pred_A['fog_test.class'].values
# df_original.loc[df_original['fog_test.stn_id'].str[0] == 'B', 'fog_test.class'] = pred_B['class'].values
df_original.loc[df_original['fog_test.stn_id'].str[0] == 'C', 'fog_test.class'] = pred_C['class'].values
# df_original.loc[df_original['fog_test.stn_id'].str[0] == 'D', 'fog_test.class'] = pred_D['class'].values
df_original.loc[df_original['fog_test.stn_id'].str[0] == 'E', 'fog_test.class'] = pred_E['class'].values


In [110]:
df_original

,Unnamed: 0,fog_test.month,fog_test.day,fog_test.time,fog_test.minute,fog_test.stn_id,fog_test.ws10_deg,fog_test.ws10_ms,fog_test.ta,fog_test.re,fog_test.hm,fog_test.sun10,fog_test.ts,fog_test.class
fog_test.year,,,,,,,,,,,,,,
L,1,1,1,0,0,AI,329.5,0.6,-2.5,0.0,87.5,0.0,-1.6,NaN
L,2,1,1,0,10,AI,321.8,1.2,-2.5,0.0,88.2,0.0,-1.6,NaN
L,3,1,1,0,20,AI,0.4,0.4,-2.5,0.0,88.6,0.0,-1.7,NaN
L,4,1,1,0,30,AI,323.6,0.7,-2.6,0.0,88.7,0.0,-1.6,NaN
L,5,1,1,0,40,AI,208.4,0.2,-2.6,0.0,89.0,0.0,-1.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L,262796,12,31,23,10,ED,338.8,6.1,4.5,0.0,71.3,0.0,1.9,4.0
L,262797,12,31,23,20,ED,335.2,5.8,4.6,0.0,71.4,0.0,1.9,4.0
L,262798,12,31,23,30,ED,337.1,5.3,4.5,0.0,72.1,0.0,1.8,4.0


In [111]:
# 혹시 결측값 있으면 4로 채움 
df_original['fog_test.class'].fillna(4, inplace=True)

In [112]:
df_original['fog_test.class'].value_counts()

4.0    262519
1.0       205
3.0        50
2.0        26
Name: fog_test.class, dtype: int64

In [102]:
df_original.to_csv('240371.csv', index=True)